# LLM with Function Calling

Example Usage: | makeresults
| fit MLTKContainer algo=llm_rag_function_calling prompt="Search Splunk for index _internal and sourcetype splunkd for events containing keyword error from 60 minutes ago to 30 minutes ago. Tell me how many events occurred" model_name="llama3" func1=1 func2=0  _time into app:llm_rag_function_calling as RAG

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [2]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import numpy as np
import pandas as pd
import os
import pymilvus
from pymilvus import (
    connections,
    utility,
    FieldSchema, CollectionSchema, DataType,
    Collection,
)
import llama_index
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Document, StorageContext, ServiceContext
from llama_index.vector_stores.milvus import MilvusVectorStore
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import textwrap
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate
from llama_index.core.agent import FunctionCallingAgentWorker
from typing import Sequence, List
from llama_index.core.tools import BaseTool, FunctionTool
from llama_index.core.agent import AgentRunner, ReActAgentWorker
from pydantic import Field
# ...
# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

def search_splunk_events(
    index: str, 
    sourcetype: str, 
    earliest_time: str, 
    latest_time: str, 
    source: str = None, 
    keyword: str =None
):
    '''
    Description on input fields
    earliest_time: Time specifier for earliest event to search, formatted like '[+|-]<time_integer><time_unit>@<time_unit>'. For example, '-12h@h' for the past 12 hours, '-5m@m' for the last 5 minutes and '-40s@s' for the last 40 seconds
    latest_time: Time specifier for latest event search, formatted like '[+|-]<time_integer><time_unit>@<time_unit>'. For example, '-12h@h' for the past 12 hours, '-5m@m' for the last 5 minutes and '-40s@s' for the last 40 seconds. For searching events up to now, set this field to 'now'
    '''
    # Imports
    import splunklib.client as splunk_client
    import splunklib.results as splunk_results
    import time 
    import pandas as pd
    # Load Splunk server info and create service
    token = os.environ["splunk_access_token"]
    host = os.environ["splunk_access_host"]
    port = os.environ["splunk_access_port"]
    service = splunk_client.connect(host=host, port=port, token=token)
    if index is not None:
        index = index
    else:
        index= ' *'
    if source is not None:
        source = source
    else:
        source = ' *'
    if sourcetype is not None:
        sourcetype = sourcetype
    else:
        sourcetype = ' *'
    if keyword is not None:
        keyword = keyword
    else:
        keyword = ' *'
    if earliest_time is not None:
        earliest = earliest_time
    else:
        earliest = '-24h@h'
    if latest_time is not None:
        latest = latest_time
    else:
        latest = "now"

    query = f"index={index} sourcetype={sourcetype} source={source} {keyword} earliest={earliest} latest={latest}"
    query_cleaned = query.strip()
    # add search keyword before the SPL
    query_cleaned="search "+query_cleaned
    
    job = service.jobs.create(
        query_cleaned,
        earliest_time=earliest, 
        latest_time=latest, 
        adhoc_search_level="smart",
        search_mode="normal")
    while not job.is_done():
        time.sleep(0.1)
    resultCount = int(job.resultCount)
    diagnostic_messages = []
    resultset = []
    processed = 0
    offset = 0
    while processed < resultCount:
        for event in splunk_results.JSONResultsReader(job.results(output_mode='json', offset=offset, count=0)):
            if isinstance(event, splunk_results.Message):
                # Diagnostic messages may be returned in the results
                diagnostic_messages.append(event.message)
                #print('%s: %s' % (event.type, event.message))
            elif isinstance(event, dict):
                # Normal events are returned as dicts
                resultset.append(event['_raw'])
                #print(result)
            processed += 1
        offset = processed   
    results = f'The list of events searched from Splunk is {str(resultset)}'
    return results

# Milvus search function
def search_record_from_vector_db(log_message: str, collection_name: str):
    from pymilvus import connections, Collection
    from llama_index.embeddings.huggingface import HuggingFaceEmbedding
    transformer_embedder = HuggingFaceEmbedding(model_name='all-MiniLM-L6-v2')
    connections.connect("default", host="milvus-standalone", port="19530")
    collection = Collection(collection_name)
    search_params = {
        "metric_type": "L2",
        "params": {"nprobe": 10},
    }
    log_message = transformer_embedder.get_text_embedding(log_message)
    results = collection.search(data=[log_message], anns_field="embeddings", param=search_params, limit=1, output_fields=["_key","label"])
    l = []
    for result in results:
        t = ""
        for r in result:
            t += f"For the log message {log_message}, the recorded similar log message is: {r.entity.get('label')}."
        l.append(t)
    return l[0]

search_splunk_tool = FunctionTool.from_defaults(fn=search_splunk_events)
search_record_from_vector_db_tool = FunctionTool.from_defaults(fn=search_record_from_vector_db)


In [4]:
# Some logging settings 
import logging
import sys
import llama_index.core
from llama_index.core.callbacks import (
    CallbackManager,
    LlamaDebugHandler,
    CBEventType,
)

llama_index.core.set_global_handler("simple")

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

## Stage 1

In [6]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

## Stage 2 - create and initialize a model

In [10]:
# initialize your model
# available inputs: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    model = {}
    model['hyperparameter'] = 42.0
    
    return model

## Stage 3 - fit the model

In [19]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model,df,param):
    info = {"message": "model trained"}
    return info

In [13]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,df,param))

{'message': 'model trained'}


## Stage 4 - apply the model

In [6]:
# apply your model
# returns the calculated results
def apply(model,df,param):
    # Example: 'all-MiniLM-L6-v2'
    query = param['options']['params']['prompt'].strip('\"')
    # Case of only two functions
    try:
        func1 = int(param['options']['params']['func1'])
        func2 = int(param['options']['params']['func2'])
    except:
        func1 = 1
        func2 = 1
    tool_list = []

    if func1:
        tool_list.append(search_splunk_tool)
    if func2:
        tool_list.append(search_record_from_vector_db_tool)
    
    try:
        model = param['options']['params']['model_name'].strip('\"')
    except:
        model="mistral"
    
    url = "http://ollama:11434"
    llm = Ollama(model=model, base_url=url, request_timeout=6000.0)

    
    worker = ReActAgentWorker.from_tools(tool_list, llm=llm)
    agent = AgentRunner(worker)     
    response = agent.chat(query)
    
    cols = {"Response": [response.response]}
    for i in range(len(response.sources)):
        if response.sources[i].tool_name != "unknown":
            cols[response.sources[i].tool_name] = [response.sources[i].content]
    result = pd.DataFrame(data=cols)
    return result

In [7]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
query = 'Search Splunk for index _internal and sourcetype splunkd for events containing keyword error pipe in the last two hours. Tell me how many events occurred'
r = apply(None,None,query)

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/usr/local/lib/python3.9/site-packages/certifi/cacert.pem'
load_verify_locations cafile='/usr/local/lib/python3.9/site-packages/certifi/cacert.pem'
DEBUG:httpcore.connection:connect_tcp.started host='ollama' port=11434 local_address=None timeout=6000.0 socket_options=None
connect_tcp.started host='ollama' port=11434 local_address=None timeout=6000.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7f1f2793b490>
connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7f1f2793b490>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
send_request_headers.complete
DEBUG:

In [106]:
cols = {"Response": [r.response]}
for i in range(len(r.sources)):
    cols[r.sources[i].tool_name] = [r.sources[i].content]
cols

{'Response': ["It seems there was a similar error related to a broken pipe that occurred on 05-12-2024 in the DSDL app sync process. The possible solution provided for this issue is to check the docker host connectivity. However, I don't have information about whether this issue has been resolved in your current situation or not. You should verify if the problem persists and whether checking docker host connectivity resolves it in your case."],
 'splunk_search_tool': ["['06-03-2024 03:45:09.104 +0000 WARN  HttpListener [3992795 webui] - Socket error from 106.72.0.128:5842 while accessing /en-US/static/@d95b3299fa65/build/api/layout-dark.js: Broken pipe']"],
 'search_record_from_vector_db': ['The recorded log message is: 05-12-2024 02:56:00.950 +0000 WARN  HttpListener [3906 HTTPDispatch] - Socket error from 127.0.0.1:33540 while accessing /services/mltk-container/sync: Broken pipe. The recorded explanation of this message is Socket error in DSDL app sync.. The recorded solution of this

In [107]:
pd.DataFrame(data=cols)

,Response,splunk_search_tool,search_record_from_vector_db
0,It seems there was a similar error related to ...,['06-03-2024 03:45:09.104 +0000 WARN HttpList...,The recorded log message is: 05-12-2024 02:56:...


In [97]:
r.response

"It seems there was a similar error related to a broken pipe that occurred on 05-12-2024 in the DSDL app sync process. The possible solution provided for this issue is to check the docker host connectivity. However, I don't have information about whether this issue has been resolved in your current situation or not. You should verify if the problem persists and whether checking docker host connectivity resolves it in your case."

In [103]:
r.sources[0].tool_name

'splunk_search_tool'

In [104]:
r.sources[1].content

'The recorded log message is: 05-12-2024 02:56:00.950 +0000 WARN  HttpListener [3906 HTTPDispatch] - Socket error from 127.0.0.1:33540 while accessing /services/mltk-container/sync: Broken pipe. The recorded explanation of this message is Socket error in DSDL app sync.. The recorded solution of this log message is Check docker host connectivity.. The past issue has been Resolved. DISTANCE: 1.027364730834961'

## Stage 5 - save the model

In [16]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    with open(MODEL_DIRECTORY + name + ".json", 'w') as file:
        json.dump(model, file)
    return model

## Stage 6 - load the model

In [17]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    model = {}
    with open(MODEL_DIRECTORY + name + ".json", 'r') as file:
        model = json.load(file)
    return model

## Stage 7 - provide a summary of the model

In [18]:
# return a model summary
def summary(model=None):
    returns = {"version": {"numpy": np.__version__, "pandas": pd.__version__} }
    return returns

After implementing your fit, apply, save and load you can train your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| fit MLTKContainer algo=barebone s from feature_* into app:barebone_model<br>

Or apply your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| apply barebone_model as the_meaning_of_life

## Send data back to Splunk HEC
When you configured the Splunk HEC Settings in the DSDL app you can easily send back data to an index with [Splunk's HTTP Event Collector (HEC)](https://docs.splunk.com/Documentation/Splunk/latest/Data/UsetheHTTPEventCollector). Read more about data formats and options in the [documentation](https://docs.splunk.com/Documentation/Splunk/latest/Data/FormateventsforHTTPEventCollector#Event_metadata).

### Use cases
- you want to offload longer running, possibly distributed computations that need to deliver results asynchroneously back into Splunk. 
- you might not want to present results back into the search pipeline after your `| fit` or `| apply` command. 
- you can easily utilize this approach for any logging purposes or other profiling tasks in your ML code so you can actively monitor and analyze your processes.

### Example

In [18]:
from dsdlsupport import SplunkHEC as SplunkHEC
hec = SplunkHEC.SplunkHEC()

In [19]:
# example to send 10 hello world events
response = hec.send_hello_world(10)

In [20]:
print("HEC endpoint %s \nreturned with status code %s \nand response message: %s" % (response.url, response.status_code, response.text))

HEC endpoint http://host.docker.internal:8088/services/collector/event 
returned with status code 200 
and response message: {"text":"Success","code":0}


In [21]:
# example to send a JSON object, e.g. to log some data
from datetime import datetime
response = hec.send({'event': {'message': 'operation done', 'log_level': 'INFO' }, 'time': datetime.now().timestamp()})

In [22]:
print("HEC endpoint %s \nreturned with status code %s \nand response message: %s" % (response.url, response.status_code, response.text))

HEC endpoint http://host.docker.internal:8088/services/collector/event 
returned with status code 200 
and response message: {"text":"Success","code":0}


## End of Stages
All subsequent cells are not tagged and can be used for further freeform code